This file creates the json files we will need for our algorithm.
If you change any of the data_files, you will need to run this file again to update the json files.

In [8]:
import csv
import json
import data_processing

In [9]:
with open('data/recipes.csv', 'r') as f:
    recipes = list(csv.DictReader(f))

url_mapping = {}
with open('data/recipes_urls.csv', 'r') as urls_file:
    reader = csv.DictReader(urls_file)
    for row in reader:
        url_mapping[row['RecipeId']] = row['Url']

In [10]:
recipes = recipes[:10000]

In [11]:
# Pre-compute the inverted index, idf, recipe norms, and id to recipe mapping
inv_idx = data_processing.build_inverted_index(recipes)
id_to_recipe = data_processing.build_id_to_recipe(recipes)
idf = data_processing.build_idf(inv_idx, len(recipes))
recipe_norms = data_processing.build_recipe_norms(inv_idx, idf)

In [12]:
# Save all the pre-computed data to four json files in /data
with open('data/inv_idx.json', 'w') as f:
    json.dump(inv_idx, f)

with open('data/id_to_recipe.json', 'r') as json_file:
    id_to_recipe = json.load(json_file)

# Step 3: Add URLs to the JSON object based on RecipeId
for recipe_id, recipe in id_to_recipe.items():
    if recipe_id in url_mapping:
        recipe['Url'] = url_mapping[recipe_id]

# Step 4: Save the updated object back to id_to_recipe.json
with open('data/id_to_recipe.json', 'w') as f:
    json.dump(id_to_recipe, f)

with open('data/idf.json', 'w') as f:
    json.dump(idf, f)

with open('data/recipe_norms.json', 'w') as f:
    json.dump(recipe_norms, f)

In [13]:
with open('data/recipes_urls.csv', 'r') as f:
    recipes_urls = list(csv.DictReader(f))

recipes_urls = recipes_urls[:10000]

with open('data/recipe_urls.json', 'w') as f:
    json.dump(recipes_urls, f)